# Ejercicio 8: Bases de Datos Vectoriales

Las bases de datos vectoriales permiten almacenar y recuperar información representada como vectores en espacios de alta dimensión. Primero vamos a revisar los fundamentos matemáticos en los que se basan.

## 1. Espacios Vectoriales

Cada documento, imagen, o consulta se representa como un vector real en un espacio ℝ^n:

$\[ \vec{d} = [d_1, d_2, \dots, d_n] \in \mathbb{R}^n \]$

Donde $\( n \)$ suele ser 384, 768 o 1536, dependiendo del modelo de embeddings utilizado.

In [2]:
import numpy as np

# Simulamos 3 documentos como vectores en R^3
doc1 = np.array([0.2, 0.1, 0.5])
doc2 = np.array([-0.1, 0.4, 0.3])
query = np.array([0.1, 0.3, 0.4])

print("Documentos:", doc1, doc2)
print("Consulta:", query)

Documentos: [0.2 0.1 0.5] [-0.1  0.4  0.3]
Consulta: [0.1 0.3 0.4]


## 2. Medidas de Similitud

El principio básico de una base vectorial es buscar elementos cuyo vector esté "cerca" del vector de consulta. Existen varias formas de medir esta cercanía:

### a. Distancia Euclidiana (L2)

$\[ \text{dist}(⇡\vec{q}, \vec{d}) = \sqrt{\sum_{i=1}^n (q_i - d_i)^2} \]$

Utilizada cuando los vectores no están normalizados. Implementada por defecto en `FAISS` con `IndexFlatL2`.

### b. Similitud Coseno

$\[ \cos(\theta) = \frac{\vec{q} \cdot \vec{d}}{\|\vec{q}\| \cdot \|\vec{d}\|} \]$

Esta métrica es ideal cuando se desea medir ángulos (dirección) en lugar de magnitudes. Se usa en `ChromaDB` y también puede simularse en FAISS si los vectores están normalizados.

Existe una relación entre ambas (cuando los vectores están normalizados):
$\[ \text{dist}_{\text{L2}}^2 = 2 - 2 \cdot \cos(\theta) \]$

In [3]:
from numpy.linalg import norm

dist1 = norm(query - doc1)
dist2 = norm(query - doc2)

print("Distancia Euclidiana a doc1:", dist1)
print("Distancia Euclidiana a doc2:", dist2)

def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

sim1 = cosine_similarity(query, doc1)
sim2 = cosine_similarity(query, doc2)

print("Similitud coseno con doc1:", sim1)
print("Similitud coseno con doc2:", sim2)

Distancia Euclidiana a doc1: 0.2449489742783178
Distancia Euclidiana a doc2: 0.24494897427831785
Similitud coseno con doc1: 0.8951435925492909
Similitud coseno con doc2: 0.8846153846153845


## 3. Normalización de Vectores

Muchos sistemas normalizan los vectores para que su norma sea 1:

$\[ \hat{\vec{v}} = \frac{\vec{v}}{\|\vec{v}\|} \]$

Esto transforma la distancia Euclidiana en una función directa de la similitud coseno, facilitando búsquedas eficientes y comparables.

In [4]:
def normalize(v):
    return v / norm(v)

q_norm = normalize(query)
d1_norm = normalize(doc1)
d2_norm = normalize(doc2)

print("Vector normalizado q:", q_norm)
print("Similitud coseno post-normalización (dot):", np.dot(q_norm, d1_norm), np.dot(q_norm, d2_norm))

# Relación teórica: dist² = 2 - 2cos(θ)
dot = np.dot(q_norm, d1_norm)
euclidean_sq = norm(q_norm - d1_norm)**2
print("2 - 2cos(theta):", 2 - 2 * dot)
print("Distancia euclidiana al cuadrado:", euclidean_sq)

Vector normalizado q: [0.19611614 0.58834841 0.78446454]
Similitud coseno post-normalización (dot): 0.895143592549291 0.8846153846153845
2 - 2cos(theta): 0.20971281490141802
Distancia euclidiana al cuadrado: 0.20971281490141774


## 4. Indexación y Aceleración

Buscar en millones de vectores directamente es costoso $(\( O(n \cdot d) \))$. Se usan estructuras aproximadas para acelerar:

### a. IVF (Inverted File Index)
- Aplica clustering (K-means) a los vectores.
- Durante la búsqueda, se consulta solo un subconjunto de clústeres.

### b. HNSW (Hierarchical Navigable Small World)
- Construye un grafo jerárquico de vecinos más cercanos.
- Permite búsquedas logarítmicas eficientes.